In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import random
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from sklearn.preprocessing import LabelEncoder

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Tokenizing and lemmatizing
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# Load intents data
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Process intents
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents in the corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Output statistics
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

# Save words and classes
pickle.dump(words, open('texts.pkl', 'wb'))
pickle.dump(classes, open('labels.pkl', 'wb'))

from google.colab import drive
drive.mount('/content/drive')

# Create training data
training = []
output_empty = [0] * len(classes)

# Create training set
for doc in documents:
    # Create the bag of words
    bag = [1 if w in doc[0] else 0 for w in words]

    # Output is a binary array corresponding to the intent
    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1

    # Print lengths for debugging
    print(f"Bag length: {len(bag)}, Output row length: {len(output_row)}")

    # Ensure that bag and output_row have correct shapes
    if len(bag) == len(words) and len(output_row) == len(classes):
        training.append([bag, output_row])
    else:
        print("Mismatch in lengths: bag -", len(bag), "output_row -", len(output_row))

# Shuffle and convert to NumPy array
random.shuffle(training)

# Reshape training data to have consistent shapes
# Use a list comprehension to extract the bag and output_row for each sample
train_x = np.array([x[0] for x in training])
train_y = np.array([x[1] for x in training])

print("Training data created")

# Create model - 3 layers
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

# Compile model
# Use learning_rate instead of lr
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fitting and saving the model
hist = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('model.h5')
print("Model created")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


283 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0080 - loss: 4.8630
Epoch 2/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0884 - loss: 4.7927
Epoch 3/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0380 - loss: 4.6737
Epoch 4/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0471 - loss: 4.5664
Epoch 5/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0265 - loss: 4.5246
Epoch 6/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0768 - loss: 4.3828
Epoch 7/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0778 - loss: 4.2773
Epoch 8/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1057 - loss: 4.1615
Epoch 9/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0877 - loss: 4.0938
Epoch 10/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1450 - loss: 3.8361
Epoch 11/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1201 - loss: 3.7462
Epoch 12/200
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy:

Model created


In [4]:
!pip install flask flask-ngrok

In [5]:
!pip install pyngrok

In [7]:
import nltk
nltk.download('popular')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import tensorflow as tf
from keras.models import load_model
import json
import random

# Load your model (make sure the model is available in the same directory or specify path)
model = load_model('model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('texts.pkl','rb'))
classes = pickle.load(open('labels.pkl','rb'))

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    if ints:
        tag = ints[0]['intent']
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
        return result
    else:
        return "Sorry, I didn't understand that."

def chatbot_response(msg):
    res = getResponse(predict_class(msg, model), intents)
    return res

from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

app = Flask(__name__, static_folder='static')

# Ngrok authentication token (make sure to set it properly)
ngrok.set_auth_token('2oi2citdQ49WW4cecn5POhcfBrk_64xfxeUrghnUs2WzvqCaU')

# Open a tunnel on the default port 5000
public_url = ngrok.connect(5000)
# Print the public URL
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    print("get_bot_response:- " + userText)

    # Directly get the response in English without language detection
    chatbot_response_text = chatbot_response(userText)

    return chatbot_response_text

# Run the Flask app
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

 * ngrok tunnel "NgrokTunnel: "https://5b68-34-27-241-225.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:10:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:10:08] "GET /static/js/bot.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:10:08] "GET /static/styles/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:10:09] "GET /static/img/mhcicon.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:11:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:11:25] "GET /static/styles/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:11:25] "GET /static/js/bot.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:11:25] "GET /static/img/mhcicon.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:12:04] "GET / HTTP/1.1" 200 -
INFO:wer

get_bot_response:- hii
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step


INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:12:11] "GET /get?msg=hii HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:12:11] "GET /static/img/mhcicon.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:13:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:13:42] "GET /static/styles/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:13:42] "GET /static/js/bot.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:13:43] "GET /static/img/mhcicon.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:14:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:14:01] "GET /static/styles/style.css HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:14:01] "GET /static/js/bot.js HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:14:02] "GET /static/img/mhcicon.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:14:10] "GET /static/img/person.png HTTP/1.1" 304 -


...

INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:26:08] "GET /get?msg=hi HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:26:08] "GET /static/img/mhcicon.png HTTP/1.1" 304 -


get_bot_response:- feeling lonely
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:26:19] "GET /get?msg=feeling%20lonely HTTP/1.1" 200 -


get_bot_response:- i don't know
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


INFO:werkzeug:127.0.0.1 - - [12/Nov/2024 15:26:33] "GET /get?msg=i%20don't%20know HTTP/1.1" 200 -


from google.colab import drive
drive.mount('/content/drive')